In [2]:
import requests
import bs4
import re

In [13]:
url = 'http://www.rating.unecon.ru/index.php?&y=2018&f=1&up=12020&s=3&g=all&upp=all&ball=hide&sort=fio'
brs_HTML = requests.get(url)
brs_Soup = bs4.BeautifulSoup(brs_HTML.text)

In [19]:
clear_HTML_re = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
def clear_HTML(text):
    return re.sub(clear_HTML_re, '', text)

### Находим названия предметов

In [15]:
subjects_short = brs_Soup.find('div', {'class': 'page_info'}).find_all('b')
subjects_short = [subject.text.strip('</b>') for subject in subjects_short]
subjects_full = brs_Soup.find('div', {'class': 'page_info'}).find_all('li')
subjects_full = [re.findall("</b> - (.*?)</li>", str(subject))[0] for subject in subjects_full]
subjects = dict(zip(subjects_short, subjects_full))
subjects

{'ДИЯ': 'Деловой иностранный язык (Экзамен)',
 'ДУ': 'Дифференциальные уравнения (Экзамен)',
 'МАIII': 'Математический анализ (I,II,III) (Экзамен)',
 'ОС': 'Операционные системы (дифф.зач.)',
 'С': 'Социология (Зачет)',
 'ФКиСЭД': 'Физическая культура и спорт (элективная дисциплина) (Зачет)',
 'Э': 'Экономика (дифф.зач.)',
 'ЯиМП': 'Языки и методы программирования (Экзамен)'}

### Получаем всех студентов

In [20]:
students = brs_Soup.find('tbody').find_all('tr')
students = {clear_HTML(student.find('a').text): [int(clear_HTML(score.text)) if clear_HTML(score.text) else 0 for score in student.find_all('td', {'class': 'w50 no_mobile'})] for student in students if student.find('a')}
students

{'Александрова Софья Викторовна': [22, 0, 30, 26, 97, 0, 57, 10],
 'Беляев Арсений Александрович': [26, 0, 35, 26, 98, 0, 56, 10],
 'Буточников Александр Валерьевич': [20, 0, 20, 0, 50, 0, 57, 10],
 'Ерофеевский Александр Сергеевич': [24, 0, 35, 26, 53, 0, 59, 20],
 'Журба Мария Владимировна': [21, 0, 25, 0, 53, 0, 49, 10],
 'Захаров Кирилл Александрович': [23, 0, 33, 26, 49, 0, 57, 20],
 'Зотов Максим Дмитриевич': [27, 0, 30, 26, 48, 0, 52, 20],
 'Кадыров Алексей Алексеевич': [22, 0, 35, 26, 35, 0, 55, 10],
 'Лепёхина Анфиса Владимировна': [24, 0, 35, 26, 96, 0, 64, 30],
 'Моннар Иван Сергеевич': [22, 0, 24, 26, 98, 0, 58, 20],
 'Мотылева Екатерина Дмитриевна': [29, 0, 35, 26, 100, 0, 59, 30],
 'Никифорова Елена Осамовна': [21, 0, 25, 26, 58, 0, 47, 20],
 'Новоселова Наталья Алексеевна': [23, 0, 35, 26, 39, 0, 59, 20],
 'Осипов Никита Сергеевич': [19, 0, 30, 26, 83, 0, 57, 30],
 'Скибин Максим Витальевич': [22, 0, 29, 26, 97, 0, 50, 10],
 'Смирнова Анастасия Эдуардовна': [25, 0, 35, 2

### Берем только нужных людей + добавляем сокращения имен

In [21]:
participants = [
    'Ерофеевский Александр Сергеевич', 
    'Захаров Кирилл Александрович', 
    'Зотов Максим Дмитриевич', 
    'Мотылева Екатерина Дмитриевна', 
    'Соболев Даниил Дмитриевич', 
    'Спирин Владислав Сергеевич', 
    'Толдов Кирилл Алексеевич', 
    'Шульняев Сергей Андреевич'
]
participants_scores = {student: students[student] for student in students if student in participants}
participants_scores

{'Ерофеевский Александр Сергеевич': [24, 0, 35, 26, 53, 0, 59, 20],
 'Захаров Кирилл Александрович': [23, 0, 33, 26, 49, 0, 57, 20],
 'Зотов Максим Дмитриевич': [27, 0, 30, 26, 48, 0, 52, 20],
 'Мотылева Екатерина Дмитриевна': [29, 0, 35, 26, 100, 0, 59, 30],
 'Соболев Даниил Дмитриевич': [23, 0, 26, 26, 33, 0, 51, 20],
 'Спирин Владислав Сергеевич': [27, 0, 25, 26, 100, 0, 49, 10],
 'Толдов Кирилл Алексеевич': [21, 0, 30, 0, 20, 0, 52, 10],
 'Шульняев Сергей Андреевич': [22, 0, 18, 26, 29, 0, 62, 10]}

In [22]:
participants_nicknames = {
    'Ерофеевский Александр Сергеевич': 'Саня', 
    'Захаров Кирилл Александрович': 'Кирюша', 
    'Зотов Максим Дмитриевич': 'Максибон',
    'Мотылева Екатерина Дмитриевна': 'Катя', 
    'Соболев Даниил Дмитриевич': 'Даня', 
    'Спирин Владислав Сергеевич': 'Владик', 
    'Толдов Кирилл Алексеевич': 'Кирилл', 
    'Шульняев Сергей Андреевич': 'Сергей'
}
inverse_participants_nicknames = {short: full for full, short in participants_nicknames.items()}
inverse_participants_nicknames

{'Саня': 'Ерофеевский Александр Сергеевич',
 'Кирюша': 'Захаров Кирилл Александрович',
 'Максибон': 'Зотов Максим Дмитриевич',
 'Катя': 'Мотылева Екатерина Дмитриевна',
 'Даня': 'Соболев Даниил Дмитриевич',
 'Владик': 'Спирин Владислав Сергеевич',
 'Кирилл': 'Толдов Кирилл Алексеевич',
 'Сергей': 'Шульняев Сергей Андреевич'}

### Генерируем текст для сообщения со статистикой по предмету(ам)

In [23]:
def one_subject_stats(name):
    index = subjects_short.index(name)
    text = subjects[name] + '\n'
    for participant in participants_scores:
        text += participants_nicknames[participant] + ': ' + str(participants_scores[participant][index]) + '\n'
    return(text[:-1])
print(one_subject_stats('МАIII'))

Математический анализ (I,II,III) (Экзамен)
Саня: 35
Кирюша: 33
Максибон: 30
Катя: 35
Даня: 26
Владик: 25
Кирилл: 30
Сергей: 18


In [24]:
def all_stats():
    text = 'Статистика по БРС' + '\n\n'
    for subject in subjects_short:
            text += one_subject_stats(subject) + '\n\n'
    return text[:-2]
print(all_stats())

Статистика по БРС

Деловой иностранный язык (Экзамен)
Саня: 24
Кирюша: 23
Максибон: 27
Катя: 29
Даня: 23
Владик: 27
Кирилл: 21
Сергей: 22

Дифференциальные уравнения (Экзамен)
Саня: 0
Кирюша: 0
Максибон: 0
Катя: 0
Даня: 0
Владик: 0
Кирилл: 0
Сергей: 0

Математический анализ (I,II,III) (Экзамен)
Саня: 35
Кирюша: 33
Максибон: 30
Катя: 35
Даня: 26
Владик: 25
Кирилл: 30
Сергей: 18

Операционные системы (дифф.зач.)
Саня: 26
Кирюша: 26
Максибон: 26
Катя: 26
Даня: 26
Владик: 26
Кирилл: 0
Сергей: 26

Социология (Зачет)
Саня: 53
Кирюша: 49
Максибон: 48
Катя: 100
Даня: 33
Владик: 100
Кирилл: 20
Сергей: 29

Физическая культура и спорт (элективная дисциплина) (Зачет)
Саня: 0
Кирюша: 0
Максибон: 0
Катя: 0
Даня: 0
Владик: 0
Кирилл: 0
Сергей: 0

Экономика (дифф.зач.)
Саня: 59
Кирюша: 57
Максибон: 52
Катя: 59
Даня: 51
Владик: 49
Кирилл: 52
Сергей: 62

Языки и методы программирования (Экзамен)
Саня: 20
Кирюша: 20
Максибон: 20
Катя: 30
Даня: 20
Владик: 10
Кирилл: 10
Сергей: 10


### Генерируем текст для статистики по человеку

In [25]:
def one_student_stats(name):
    full_name = inverse_participants_nicknames[name]
    text = name + '\n'
    for i in range(len(participants_scores[full_name])):
        text += subjects[subjects_short[i]] + ': ' + str(participants_scores[full_name][i]) + '\n'
    return text[:-1]
print(one_student_stats('Саня'))

Саня
Деловой иностранный язык (Экзамен): 24
Дифференциальные уравнения (Экзамен): 0
Математический анализ (I,II,III) (Экзамен): 35
Операционные системы (дифф.зач.): 26
Социология (Зачет): 53
Физическая культура и спорт (элективная дисциплина) (Зачет): 0
Экономика (дифф.зач.): 59
Языки и методы программирования (Экзамен): 20
